In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
with open('sx-stackoverflow-a2q.txt','r') as f:
    lines = f.readlines()

In [3]:
def dateparse(time_in_secs):
    return pd.to_datetime(time_in_secs, unit = 's')

In [4]:
data = pd.read_csv('sx-stackoverflow-a2q.txt', delimiter=' ', header=None)

In [5]:
data.columns = ['Source', 'Destination', 'Time']

In [24]:
data

,Source,Destination,Time
0,9,8,1217567877
1,1,1,1217573801
2,13,1,1217606247
3,17,1,1217617639
4,48,2,1217618182
...,...,...,...
17823520,2773607,1048138,1457266581
17823521,6018278,1982354,1457266589
17823522,3187183,1404306,1457266650
17823523,6022341,1667278,1457266660


In [6]:
g1 = pd.read_csv('sx-stackoverflow-a2q.txt', delimiter=' ', header=None)
g2 = pd.read_csv('sx-stackoverflow-c2a.txt', delimiter=' ', header=None)
g3 = pd.read_csv('sx-stackoverflow-c2q.txt', delimiter=' ', header=None)

In [26]:
df1 = pd.DataFrame(g1)
df2 = pd.DataFrame(g2)
df3 = pd.DataFrame(g3)

In [24]:
df1.shape
df1.drop([20001,], axis = 0)

SyntaxError: invalid syntax (<ipython-input-24-978294b2661d>, line 2)

In [25]:
df1.columns = ['SourceAns', 'DestinationAns', 'TimeAns']
df2.columns = ['SourceCommA', 'DestinationCommA', 'Time']
df3.columns = ['SourceCommQ', 'DestinationCommQ', 'TimeCommQ']

In [ ]:
df = df3.merge(df1, how='left').merge(df2, how='left')
print (df)

# 4. Algorithmic question

In [28]:
# Here I create a Dictionary in order to define the nodes and their neighbours.

n_kids = { "A" : ["B", "C"],
          "B" : ["D", "E", "A"],
          "C" : ["A"],
          "D" : ["B", "E"],
          "E" : ["B", "D"]
}

# Here I create a Function in order to define the edges between the nodes.

def quarrel(n_kids):
    
    # Initialize empty list of edges
    quarrel = [] 
    
    # Initialize empty list for the first dormitory
    s1 = []
    
    # Initialize empty list for the second dormitory
    s2 = []
    
    for node in n_kids:
        for neighbour in n_kids[node]:
            if (neighbour, node) not in quarrel:
                quarrel.append((node, neighbour))
            if (neighbour, node) in quarrel:
                s1.append(node)
                s2.append(neighbour)
                
                # Even if I can take more than once a kid, I just want to know where he goes
                s1 = set(s1) 
                s2 = set(s2)
                
                # Just for taking a look at the output
                print(s1,s2)
                
                return 1
            else:
                break

print(quarrel(n_kids))

{'D'} {'B'}
1
